In [2]:
import pandas as pd
from main import result_path
import numpy as np

%load_ext autoreload
%autoreload 2

In [3]:
dataset_name = 'Adult'

methond_names = ['baseline', 'ARL', 'ImprovedModel']

# methond_names = ['baseline', 'ARL']


# for corr in [0.1, 0.2, 0.5, 0.05]:
#     for disc in [0.1, 0.2, 0.5, 0.05, 1]:
#         methond_names += [f"ImprovedModel_corr_{corr}_{disc}"]
        
        
df = pd.DataFrame()
for method_name in methond_names:
    curr_df = pd.DataFrame()
    try:
        for seed in range(20):
            
            path = result_path(dataset_name, method_name, seed)
            seed_df = pd.read_csv(path, index_col='Unnamed: 0')
            curr_df = pd.concat([curr_df, seed_df])
            
    except Exception as e:
#         print(f"read only {len(curr_df)} seeds.")
        pass

    curr_df = curr_df.mean()
    curr_df.name = (f"{dataset_name}_{method_name}")
    df = pd.concat([df, curr_df], axis=1)
    
df = df.T

features_to_display = ['Accuracy', 'min subgroup AUC', 'average subgroup AUC', 'minority subgroup AUC']
DeltaAccuracy_features = list(filter(lambda x: ' group' in x and 'DAccuracy' in x, df.columns))
features_to_display += DeltaAccuracy_features
df['avg. DAccuracy'] = df[DeltaAccuracy_features].mean(axis=1)

features_to_display += ['corr', 'Test WSC', 'avg. DAccuracy']
df['HSIC'] = np.log10(df['HSIC'])
df[features_to_display]


,Accuracy,min subgroup AUC,average subgroup AUC,minority subgroup AUC,DAccuracy sex group 0,DAccuracy sex group 1,DAccuracy race group 0,DAccuracy race group 1,DAccuracy race group 2,DAccuracy race group 3,DAccuracy race group 4,corr,Test WSC,avg. DAccuracy
Adult_baseline,0.84014,0.680344,0.878424,0.954891,0.074888,0.036944,0.062415,0.034109,0.066647,0.050302,0.007189,0.082288,0.705926,0.047499
Adult_ARL,0.83957,0.675927,0.873405,0.946924,0.075693,0.037342,0.061367,0.032572,0.067824,0.051692,0.007550,0.082761,0.711176,0.047720
Adult_ImprovedModel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# An Idea: iteratively decrease the loss of every example.
# start by decreasing all losses to 1. then to 0.5, then to 0.25 and so on...